In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.text import Tokenizer # Changed import statement to use tensorflow.keras
from tensorflow.keras.preprocessing.sequence import pad_sequences # Changed import statement to use tensorflow.keras
from tensorflow.keras.models import Sequential # Changed import statement to use tensorflow.keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D # Changed import statement to use tensorflow.keras
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # Changed import statement to use tensorflow.keras
import re

from sklearn.preprocessing import LabelEncoder

file_path = '/content/drive/My Drive/Sentiment.csv'
# Update the path to reflect the actual location of your file in Google Drive

data = pd.read_csv(file_path) # Changed to use the updated file path
# Keeping only the neccessary columns
data = data[['text','sentiment']]

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model
# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

<ipython-input-7-8f88e2e1f757>:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt', ' ')
<ipython-input-7-8f88e2e1f757>:24: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace('rt', ' ')


291/291 - 61s - loss: 0.8199 - accuracy: 0.6493 - 61s/epoch - 209ms/step
144/144 - 4s - loss: 0.7939 - accuracy: 0.6741 - 4s/epoch - 27ms/step
0.7939354777336121
0.67409348487854
['loss', 'accuracy']


In [8]:
model.save('sentimentAnalysis.h5')


In [9]:
from keras.models import load_model
model = load_model('sentimentAnalysis.h5')
new_text = ["A lot of good things are happening. We are respected again throughout the world, and that's a great thing .@realDonaldTrump"]

# 1. Tokenize the new text using the same tokenizer used during training
new_text_sequences = tokenizer.texts_to_sequences(new_text)

# 2. Pad the sequences to match the input shape expected by the model
new_text_padded = pad_sequences(new_text_sequences, maxlen=X.shape[1]) # Use X.shape[1] from training data

# 3. Now you can predict using the processed input
predictions = model.predict(new_text_padded)

print(predictions)


1/1 [==============================] - 0s 371ms/step
[[0.7095763  0.09114243 0.19928129]]


In [10]:
!pip install tensorflow==2.12.0
!pip install keras==2.12.0
!pip install keras.utils


In [11]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import LSTM

# Function to create the model, as it's required by KerasClassifier
def create_model(lstm_out=196, dropout=0.2):
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(LSTM(lstm_out, dropout=dropout, recurrent_dropout=dropout))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

batch_size1 = [10, 20, 40]
epochs1 = [1, 2, 3]

# Define the grid of parameters to search
param_grid = dict(batch_size=batch_size1, epochs=epochs1)

# Create GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, Y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

<ipython-input-11-3e27ad9c23bd>:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


Best: 0.679435 using {'batch_size': 40, 'epochs': 2}
